In [ ]:
# 1 from typing import List
from sentence_transformers import SentenceTransformer
import faiss
import logging

class RetrievalComponent:
    def __init__(self, config):
        self.config = config
        self.documents = []
        self.index = None
        self.retriever = None
        self.logger = logging.getLogger(__name__)

    def load_knowledge_base(self, file_path: str):
        try:
            with open(file_path, 'r') as f:
                self.documents = [line.strip() for line in f if line.strip()]
            self.logger.info(f"Loaded {len(self.documents)} documents")

            # Create embeddings index
            embed_model = SentenceTransformer(self.config['retriever_model'])
            embeddings = embed_model.encode(self.documents)
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
            self.index.add(embeddings.astype('float32'))
            self.retriever = embed_model

        except Exception as e:
            self.logger.error(f"Knowledge base error: {str(e)}")
            raise

    def retrieve(self, query: str, k: int = 3) -> List[str]:
        try:
            query_embed = self.retriever.encode([query])
            distances, indices = self.index.search(query_embed.astype('float32'), k)
            return [self.documents[i] for i in indices[0]]
        except Exception as e:
            self.logger.error(f"Retrieval error: {str(e)}")
            raise


In [6]:
# 2

from typing import List
from sentence_transformers import SentenceTransformer
import faiss
import logging

class RetrievalComponent:
    def __init__(self, config):
        self.config = config
        self.documents = []
        self.index = None
        self.retriever = None
        self.logger = logging.getLogger(__name__)

    def load_knowledge_base(self, file_path: str):
        try:
            with open(file_path, 'r') as f:
                self.documents = [line.strip() for line in f if line.strip()]
            self.logger.info(f"Loaded {len(self.documents)} documents")

            # Create embeddings index
            embed_model = SentenceTransformer(self.config['retriever_model'])
            embeddings = embed_model.encode(self.documents)
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
            self.index.add(embeddings.astype('float32'))
            self.retriever = embed_model

        except Exception as e:
            self.logger.error(f"Knowledge base error: {str(e)}")
            raise

    def retrieve(self, query: str, k: int = 3) -> List[str]:
        try:
            query_embed = self.retriever.encode([query])
            distances, indices = self.index.search(query_embed.astype('float32'), k)
            return [self.documents[i] for i in indices[0]]
        except Exception as e:
            self.logger.error(f"Retrieval error: {str(e)}")
            raise


In [12]:
from typing import List
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.cuda.amp import autocast
import logging
from peft import LoraConfig, get_peft_model, TaskType  # Correct import for TaskType

class GenerationComponent:
    def __init__(self, config, device='cuda'):
        self.config = config
        self.tokenizer = None
        self.model = None
        self.device = device
        self.logger = logging.getLogger(__name__)

    def setup_generator(self):
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.config['model_name'], use_fast=True
            )
            base_model = AutoModelForCausalLM.from_pretrained(
                self.config['model_name'],
                torch_dtype=torch.float16,
                device_map='auto'
            )
            peft_config = LoraConfig(
                task_type=TaskType.CAUSAL_LM,
                r=8,
                lora_alpha=32,
                lora_dropout=0.1,
                target_modules=["q_proj", "v_proj"]
            )
            self.model = get_peft_model(base_model, peft_config)
            self.logger.info("Generator ready")

        except Exception as e:
            self.logger.error(f"Generator setup error: {str(e)}")
            raise

    def rag_generate(self, query: str, max_length: int = 500) -> str:
        try:
            # Retrieve relevant context
            context = self.retrieve(query)
            prompt = f"Context: {' '.join(context)}\n\nQuestion: {query}\nAnswer:"

            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=self.config['max_length']
            ).to(self.device)

            with autocast():
                outputs = self.model.generate(
                    **inputs,
                    max_length=max_length,
                    temperature=0.7,
                    do_sample=True
                )

            return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        except Exception as e:
            self.logger.error(f"Generation error: {str(e)}")
            raise

    def retrieve(self, query: str, k: int = 3) -> List[str]:
        # Placeholder for the retrieve method, should be implemented
        # This method should return a list of relevant context strings based on the query
        return []


In [13]:
# nu part a
from typing import List
from sentence_transformers import SentenceTransformer
import faiss
import logging

# Retrieval Component
class RetrievalComponent:
    def __init__(self, config):
        self.config = config
        self.documents = []
        self.index = None
        self.retriever = None
        self.logger = logging.getLogger(__name__)

    def load_knowledge_base(self, file_path: str):
        try:
            with open(file_path, 'r') as f:
                self.documents = [line.strip() for line in f if line.strip()]
            self.logger.info(f"Loaded {len(self.documents)} documents")

            # Create embeddings index
            embed_model = SentenceTransformer(self.config['retriever_model'])
            embeddings = embed_model.encode(self.documents)
            self.index = faiss.IndexFlatL2(embeddings.shape[1])
            self.index.add(embeddings.astype('float32'))
            self.retriever = embed_model

        except Exception as e:
            self.logger.error(f"Knowledge base error: {str(e)}")
            raise

    def retrieve(self, query: str, k: int = 3) -> List[str]:
        try:
            query_embed = self.retriever.encode([query])
            distances, indices = self.index.search(query_embed.astype('float32'), k)
            return [self.documents[i] for i in indices[0]]
        except Exception as e:
            self.logger.error(f"Retrieval error: {str(e)}")
            raise

# Generation Component with RAG Integration
class GenerationComponent:
    def __init__(self, config, retriever: RetrievalComponent, device='cuda'):
        self.config = config
        self.retriever = retriever
        self.device = device
        self.logger = logging.getLogger(__name__)

    def rag_generate(self, query: str, max_length: int = 500) -> str:
        try:
            # Retrieve relevant context
            context = self.retriever.retrieve(query)
            prompt = f"Context: {' '.join(context)}\n\nQuestion: {query}\nAnswer:"

            # Run local Ollama model
            response = run_local_ollama_model(self.config['model_name'], prompt)


SyntaxError: incomplete input (147054974.py, line 57)

In [14]:
# part 2  nu b
import subprocess
import time
import pickle
from datetime import datetime
import os

HISTORY_FILE = "ollama_history.pkl"

# History Management
def save_to_pickle(response_data):
    """Save response data to pickle file with timestamp"""
    try:
        with open(HISTORY_FILE, "rb") as f:
            history = pickle.load(f)
    except (FileNotFoundError, EOFError):
        history = {
            'created_at': datetime.now().isoformat(),
            'interactions': []
        }

    history['interactions'].append(response_data)

    with open(HISTORY_FILE, "wb") as f:
        pickle.dump(history, f)

def run_local_ollama_model(model_name, prompt, timeout=60):
    """
    Runs a local Ollama model and saves responses to timestamped pickle file.
    """
    start_time = time.time()
    response_data = {
        'timestamp': datetime.now().isoformat(),
        'model': model_name,
        'prompt': prompt,
        'response': '',
        'execution_time': 0
    }

    try:
        process = subprocess.run(
            ["ollama", "run", model_name],
            input=prompt,
            capture_output=True,
            text=True,
            timeout=timeout
        )

        response = process.stdout.strip()
        error = process.stderr.strip()
        elapsed_time = time.time() - start_time

        if error:
            response_data['response'] = f"Error: {error}"
        else:
            response_data.update({
                'response': response,
                'execution_time': round(elapsed_time, 2)
            })

        save_to_pickle(response_data)

        print(f"Execution time: {elapsed_time:.2f} seconds")
        return response

    except subprocess.TimeoutExpired:
        response_data['response'] = "Error: Response timed out."
        save_to_pickle(response_data)
        return response_data['response']

def load_history():
    """Load and print interaction history"""
    try:
        with open(HISTORY_FILE, "rb") as f:
            history = pickle.load(f)
            print(f"History created at: {history['created_at']}")
            for idx, interaction in enumerate(history['interactions'], 1):
                print(f"\nInteraction {idx} ({interaction['timestamp']}):")
                print(f"Model: {interaction['model']}")
                print(f"Prompt: {interaction['prompt']}")
                print(f"Response: {interaction['response']}")
                print(f"Execution time: {interaction['execution_time']}s")
    except FileNotFoundError:
        print("No history found")

# Example usage
if __name__ == "__main__":
    # Configuration
    config = {
        'model_name': "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L",
        'retriever_model': "sentence-transformers/all-MiniLM-L6-v2",
        'max_length': 512
    }

    # Initialize components
    retriever = RetrievalComponent(config)
    retriever.load_knowledge_base("path_to_knowledge_base.txt")

    generator = GenerationComponent(config, retriever)

    # Define the prompt
    prompt_text = "Explain the benefits of running models locally."

    # Get response
    response = generator.rag_generate(prompt_text)
    print("AI Response:", response)

    # View history
    print("\n=== Interaction History ===")
    load_history()


Knowledge base error: [Errno 2] No such file or directory: 'path_to_knowledge_base.txt'


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_knowledge_base.txt'